# Tutorial - Text Mining - Classification 

We will predict the category of discussion posts in a newsgroup.

**The unit of analysis is a discussion post**

### Import common packages

In [36]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shanthi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Shanthi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [38]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('averaged_perceptron_tagger') # you only need to run this once
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

np.random_seed = 1

### Load data

In [39]:
news = pd.read_csv('news.csv')
news.shape

(597, 5)

In [40]:
news.head(5)

,TEXT,graphics,hockey,medical,newsgroup
0,I have a few reprints left of chapters from my...,1,0,0,graphics
1,"gnuplot, etc. make it easy to plot real valued...",1,0,0,graphics
2,Article-I.D.: snoopy.1pqlhnINN8k1 References: ...,1,0,0,graphics
3,"Hello, I am looking to add voice input capabil...",1,0,0,graphics
4,I recently got a file describing a library of ...,1,0,0,graphics


# Applying Lemmatisation code

In [41]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lmt = WordNetLemmatizer()

def get_wordnet_pos_tag(word):
    """WordNetLemmatizer will map the POS tag to the first character."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"A": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lmt_text(text):
    """Lemmatizing input text"""
    lmt_text = []
    for word in nltk.word_tokenize(text):
        pos = get_wordnet_pos_tag(word)
        lmt_text.append(lmt.lemmatize(word, pos=pos))
    return " ".join(lmt_text)

# Load news dataset
news = news

# Lemmatize news headline column
news["TEXT"] = news["TEXT"].apply(lmt_text)

# Print the first 5 rows of the lemmatized dataset
print(news.head())

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shanthi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shanthi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                TEXT  graphics  hockey  \
0  I have a few reprint left of chapter from my b...         1       0   
1  gnuplot , etc . make it easy to plot real valu...         1       0   
2  Article-I.D . : snoopy.1pqlhnINN8k1 References...         1       0   
3  Hello , I be look to add voice input capabilit...         1       0   
4  I recently get a file describe a library of re...         1       0   

   medical newsgroup  
0        0  graphics  
1        0  graphics  
2        0  graphics  
3        0  graphics  
4        0  graphics  


In [42]:
news["TEXT"]

0      I have a few reprint left of chapter from my b...
1      gnuplot , etc . make it easy to plot real valu...
2      Article-I.D . : snoopy.1pqlhnINN8k1 References...
3      Hello , I be look to add voice input capabilit...
4      I recently get a file describe a library of re...
                             ...                        
592    carl @ SOL1.GPS.CALTECH.EDU ( Carl J Lydick ) ...
593    In article < 1qmlgaINNjab @ hp-col.col.hp.com ...
594    Article-I.D . : kestrel.1993Apr16.172052.27843...
595    In article < 1qmlgaINNjab @ hp-col.col.hp.com ...
596    I have a 42 yr old male friend , misdiagnosed ...
Name: TEXT, Length: 597, dtype: object

### Check for missing values

In [43]:
news[['TEXT']].isna().sum()

TEXT    0
dtype: int64

## Assign the input variable to X and the target variable to y

In [44]:
X = news['TEXT']

This is a multi-class classification problem. There are three categories we will predict:<br>
Whether a post is "graphics," "hockey," or "medical" related

In [45]:
y = news['newsgroup']
y.unique()

array(['graphics', 'hockey', 'medical'], dtype=object)

In [46]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
print(le.classes_)
y = le.transform(y)
y

['graphics' 'hockey' 'medical']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Split the data

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [48]:
X_train.shape, y_train.shape

((417,), (417,))

In [49]:
X_test.shape, y_test.shape

((180,), (180,))

In [50]:
X_train.head(5)

281    MIGHTY ONES GET MIGHTIER : TPS , the Finnish C...
159    I be look for a copy of the follow Siggraph pu...
135    Our group recently bought a Mitsubishi P78U vi...
124    Has anyone get multiverse to work ? I have bui...
569    In article < 1993Apr15.173902.66278 @ cc.usu.e...
Name: TEXT, dtype: object

In [51]:
y_train[:5]

array([1, 0, 0, 0, 2])

## Sklearn: Text preparation

For simplicity (and focus), we will not do any text cleaning or preprocessing. We will just use the raw text as input to the model. See the text mining fundamentals tutorial for more details on text cleaning and preprocessing.

In [52]:
#TfidfVectorizer includes pre-processing, tokenization, filtering stop words
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X_train = tfidf_vect.fit_transform(X_train)

**Notice in the previous step that we use `fit_transform` on TRAIN. When we transform the TEST data, we need to use `transform` only. This enables us to keep the number of columns (features) the same across the data sets. Otherwise, they WILL be different, and no model will work!**

In [53]:
# Perform the TfidfVectorizer transformation
# Be careful: We are using the train fit to transform the test data set. Otherwise, the test data 
# features will be very different and match the train set!!!

X_test = tfidf_vect.transform(X_test)

In [54]:
X_train.shape, X_test.shape

((417, 8772), (180, 8772))

In [55]:
# These data sets are "sparse matrix". We can't see them unless we convert using toarray()
X_train

<417x8772 sparse matrix of type '<class 'numpy.float64'>'
	with 28465 stored elements in Compressed Sparse Row format>

In [56]:
# These data sets are "sparse matrix". We can't see them unless we convert using toarray()
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Latent Semantic Analysis (Singular Value Decomposition)

In [57]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, n_iter=10) #n_components is the number of topics, which should be less than the number of features

X_train= svd.fit_transform(X_train)
X_test = svd.transform(X_test)

In [58]:
X_train.shape, X_test.shape

((417, 300), (180, 300))

## Random Forest

In [59]:
from sklearn.ensemble import RandomForestClassifier 

rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 
_ = rnd_clf.fit(X_train, y_train)

### Evaluating Model Performance

In [68]:
from sklearn.metrics import accuracy_score

In [69]:
#Train accuracy - Not a good measure of model performance as we are using the same data set to train and test
y_pred_train = rnd_clf.predict(X_train)
acc = accuracy_score(y_train, y_pred_train)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9952


In [70]:
#Test accuracy
y_pred_test = rnd_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Train acc: {accuracy_score(y_test, y_pred_test):.4f}")

Train acc: 0.9000


In [63]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_test)

array([[58,  0,  6],
       [ 3, 49,  4],
       [ 4,  1, 55]], dtype=int64)

## Stochastic Gradient Descent Classifier

In [64]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100)
_ = sgd_clf.fit(X_train, y_train)

### Evaluating Model Performance

In [65]:
#Train accuracy
y_pred_train = sgd_clf.predict(X_train)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9976


In [66]:
#Test accuracy
y_pred_test = sgd_clf.predict(X_test)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9976


In [67]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_test)

array([[63,  0,  1],
       [ 3, 53,  0],
       [ 4,  1, 55]], dtype=int64)

## Summary
Hence from the above results it is observed that after performing lemmatisation the accuracy for the random forest and gradient 
descent classifier are increased.

In case of Random forest it is observed that the train accuracy is changed from 0.9760 to 0.9952 after lemmatisation whereas the 
test accuracy is changed from 0.9111 to 0.900 after lemmatisation

In case of stochastic Gradient Descent classifier it is observed that the train accuracy is changed from 0.9952 to 0.9976after lemmatisation whereas the test accuracy is changed from 0.9952 to 0.9976 afer lemmatisation


